# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [2]:
import pandas as pd 
import numpy as np 

url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df_origin = pd.read_csv(url)

display(df_origin.columns)
df_origin.columns = [col.strip().replace(" ", "_").lower() for col in df_origin.columns]

###
# Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

df_filtered = df_origin[(df_origin["total_claim_amount"] > 1000) & (df_origin["response"] == "Yes")]

###
#Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for 
# customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

df = df_origin[df_origin["response"] == "Yes" ][["total_claim_amount","response","policy_type","gender"]].round(2).copy()

pivot_df = df.pivot_table(index='policy_type', columns='gender', values=['total_claim_amount'], aggfunc='mean').round(2)

display(pivot_df)
# data show's special auto and personal autos have higher claim amounts, with special autos being the top ranker for females and personal autos for males.

### Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

df2_count = df_origin.groupby("state")["policy_type"].count()
df2 = df2_count[df2_count > 500]
display(df2)
# California has the maximum clients 3552 with Oregon short behind (2909). Washington is the lowest with only 888 customers.

### Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

df3 = df_origin.groupby(["education","gender"])["customer_lifetime_value"].agg(['max', 'min', 'mean']).round(2)
display(df3)

# first there's a big dispersion of data. Maximum and minimum values show sometimes over 50x the range. Master education seem to show lower dispersion.
# For most educational levels, females show higher max values (except for HS and below). 
# except for master and college, there's a big gap between genders for max/min values.



Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

total_claim_amount        
gender                          F       M
policy_type                              
Corporate Auto             433.74  408.58
Personal Auto              452.97  457.01
Special Auto               453.28  429.53

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: policy_type, dtype: int64

max      min     mean
education            gender                            
Bachelor             F       73225.96  1904.00  7874.27
                     M       67907.27  1898.01  7703.60
College              F       61850.19  1898.68  7748.82
                     M       61134.68  1918.12  8052.46
Doctor               F       44856.11  2395.57  7328.51
                     M       32677.34  2267.60  7415.33
High School or Below F       55277.45  2144.92  8675.22
                     M       83325.38  1940.98  8149.69
Master               F       51016.07  2417.78  8157.05
                     M       50568.26  2272.31  8168.83

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [3]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [15]:
# Settings
pd.set_option('display.max_rows', 80)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 20)

df_origin.head(3)

# Convert 'Effective To Date' to datetime
df_origin['effective_to_date'] = pd.to_datetime(df_origin['effective_to_date'], format='%m/%d/%y')



In [80]:
current_date = datetime.now()

# Function to calculate policy start date
def calculate_policy_start_date(effective_date, months_since_inception):
    return effective_date - relativedelta(months=months_since_inception)

df_origin["policy_start_date"] = df_origin.apply(lambda row: calculate_policy_start_date(row['effective_to_date'], row['months_since_policy_inception']), axis=1).dt.strftime('%B %Y')

df5 = df_origin[["state",'number_of_policies',"policy_start_date"]].copy()
#display(df5)

# Present the data in a table where the months are arranged as columns and the states are arranged as rows.

pivot_df5 = df5.pivot_table(index='state', columns='policy_start_date', values=['number_of_policies'], aggfunc='sum')
display(pivot_df5)

number_of_policies                                   \
policy_start_date         April 2003 April 2004 April 2005 April 2006   
state                                                                   
Arizona                           77         42         75         42   
California                        80         69        112        118   
Nevada                            27         40         54         14   
Oregon                            67         97         92         92   
Washington                        18          8         29          9   

                                                                           \
policy_start_date April 2007 April 2008 April 2009 April 2010 August 2003   
state                                                                       
Arizona                   61         64         65         55          36   
California                66        106        101        114         103   
Nevada                    23         36         22         21          18   
Oregon                   106         52         96         69          68   
Washington                35         23         22         24          32   

                                                                               \
policy_start_date August 2004 August 2005 August 2006 August 2007 August 2008   
state                                                                           
Arizona                    55          30          63          41          83   
California                 92         113         101         123          66   
Nevada                     36          14          25          35          30   
Oregon                     86          78          64          95          54   
Washington                 26          20          17          10          28   

                               ...                                         \
policy_start_date August 2009  ... October 2004 October 2005 October 2006   
state                          ...                                          
Arizona                    56  ...           60           94           56   
California                 93  ...           86          102          161   
Nevada                     22  ...           37           38           18   
Oregon                    111  ...           70           98          108   
Washington                 11  ...           14           41           45   

                                                                       \
policy_start_date October 2007 October 2008 October 2009 October 2010   
state                                                                   
Arizona                     73           72           86           56   
California                  97          116           88          173   
Nevada                      15           25           43           25   
Oregon                      70           93           35           99   
Washington                  21           22           27           23   

                                                                               \
policy_start_date September 2003 September 2004 September 2005 September 2006   
state                                                                           
Arizona                       35             43             67             41   
California                    60             55             59            153   
Nevada                        45             38             17             30   
Oregon                       120             96             71             88   
Washington                    47             12             38             19   

                                                                               
policy_start_date September 2007 September 2008 September 2009 September 2010  
state                                                                          
Arizona                       63             71             59             4

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [69]:
#  group the data by state and month, then count the number of policies sold for each group. 
#  Afterwards, you will need to sort the data by the count of policies sold in descending order.*

df6 = df5.copy()
df6_grouped = df6.groupby(["policy_start_date", "state"])["number_of_policies"].agg(["sum"]).sort_values(by="sum", ascending=False)
df6_grouped



,,sum
policy_start_date,state,
September 2008,California,187
March 2010,California,183
October 2010,California,173
October 2006,California,161
February 2009,California,154
...,...,...
August 2010,Nevada,6
May 2004,Nevada,6
December 2003,Washington,5


In [77]:
# select the top 3 states with the highest number of policies sold.*

df6_policies = df_origin.groupby("state")["number_of_policies"].sum().reset_index().sort_values(by="number_of_policies", ascending=False).head(3)
df6_list = df6_policies["state"].tolist()
display(df6_list)


['California', 'Oregon', 'Arizona']

In [78]:
# create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*
df6f = df6[df6["state"].isin(df6_list)]

pivot_df6f = df6f.pivot_table(values="number_of_policies", index="state", columns="policy_start_date", aggfunc='sum', fill_value=0)
pivot_df6f

policy_start_date,April 2003,April 2004,April 2005,April 2006,April 2007,April 2008,April 2009,April 2010,August 2003,August 2004,August 2005,August 2006,August 2007,August 2008,August 2009,...,October 2004,October 2005,October 2006,October 2007,October 2008,October 2009,October 2010,September 2003,September 2004,September 2005,September 2006,September 2007,September 2008,September 2009,September 2010
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Arizona,77,42,75,42,61,64,65,55,36,55,30,63,41,83,56,...,60,94,56,73,72,86,56,35,43,67,41,63,71,59,40
California,80,69,112,118,66,106,101,114,103,92,113,101,123,66,93,...,86,102,161,97,116,88,173,60,55,59,153,130,187,83,106
Oregon,67,97,92,92,106,52,96,69,68,86,78,64,95,54,111,...,70,98,108,70,93,35,99,120,96,71,88,119,91,101,59


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

In [121]:
df7 = df_origin[["response", 'sales_channel']]


df7_gb = df7.groupby(["response","sales_channel"])["response"].agg(["count"]).reset_index()
df_totals = df7_gb.groupby("sales_channel")["count"].sum().reset_index()
df_totals = df_totals.rename(columns={"count": "total"})

df7_gb = df7_gb.merge(df_totals, on="sales_channel")
df7_gb["Percentage Yes"] = df7_gb.apply(lambda row: (row["count"] / row["total"]) * 100 if row["response"] == "Yes" else None,axis=1)

df7_gb

# Agents had the highest count of yes by almost the double of the second. Web had the least count of yes. Interestingly, with the lowest total count, Web seem to have had a good
# answer being second rated in yes by total intervention, followed by Branch. Call center had the lowest values, but close to the Branch.


,response,sales_channel,count,total,Percentage Yes
0,No,Agent,3148,3890,NaN
1,Yes,Agent,742,3890,19.074550
2,No,Branch,2539,2865,NaN
3,Yes,Branch,326,2865,11.378709
4,No,Call Center,1792,2013,NaN
5,Yes,Call Center,221,2013,10.978639
6,No,Web,1334,1511,NaN
7,Yes,Web,177,1511,11.714097


External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9